In [1]:
import pandas as pd
from fundamentos import (
                          CalculoWACC, 
                          VariacaoReceita, 
                          ValuationModoloGordon, 
                          OutrosAtivosNaoOperacionais,
                          PassivoTotalMenosDivida,
                          NecessidadeCapitalGiro,
                          IndicadoresFinanceiros,
                          ValuationFluxoCaixaDescontado)

from dotenv import load_dotenv
import ipeadatapy as ip
from datetime import datetime
import pandas as pd
import yfinance as yf
import sidrapy
import warnings
warnings.filterwarnings('ignore')
load_dotenv()


False

# Valuation modolo gordon

In [ ]:
# Valuation modolo gordon
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
lista_valuation = []
for acao in acoes:
    acao_sa = acao + ".SA"
    valu = ValuationModoloGordon(acao_sa)
    try:
        dicionario = valu.preco_acao()
        lista_valuation.append(dicionario)
    except KeyError:
        print("Erro ao obter dados da ação:", acao_sa)
    #print(dicionario)
    
valuation_acoes = pd.DataFrame(lista_valuation)
valuation_acoes.sort_values(by='diferenca', ascending=False).head(40)

# Valuation fluxo de caixa descontado

In [ ]:
import warnings
warnings.filterwarnings('ignore')
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
for acao in acoes:
    try:
        wac = CalculoWACC(ticker=f'{acao}.SA')
        print(f'Custo Capital Proprio da acao {acao}:',wac.wacc())
    except KeyError:
        print(f'Erro ao obter dados da acao {acao}')
    except ValueError:
        print(f'Erro ao obter dados da acao {acao}')

In [ ]:
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
for acao in acoes:
    try:
        vari = VariacaoReceita(ticker=f'{acao}.SA')
        print(f'Custo Capital Proprio da acao {acao}:',vari.receita_crescimento_metricas())
    except KeyError:
        print(f'Erro ao obter dados da acao {acao}')
    except ValueError:
        print(f'Erro ao obter dados da acao {acao}')
    except ConnectionError:
        print(f'Erro ao obter dados da acao {acao}')

In [2]:
ind = IndicadoresFinanceiros(ticker='AZEV3.SA')
indicadores_petr = ind.todos_indicadores()
indicadores_petr

[*********************100%***********************]  1 of 1 completed


{'margemebit': -0.188,
 'ultimareceita': 442596000.0,
 'variacaoreceita': {'mean_deflacionada': 1.8544798993495084,
  'median_deflacionada': 1.7348590789166258,
  'mean_normal': 2.054389064389649,
  'median_normal': 2.009985902255639},
 'depreciacaocapex': 0.1,
 'capexreceita': 0.1,
 'wacc': 0.166,
 'quantidadeacoes': 187167464.0,
 'dividatotal': 39103000.0,
 'caixa': 2090000.0,
 'outrosativos': 360930000.0,
 'passivosmenosdivida': 367763000.0,
 'necessidadecapitalgiro': -104186000.0,
 'percentualimposto': 0.5714}

In [2]:
ind = IndicadoresFinanceiros(ticker='PETR4.SA')
pd.options.display.float_format = '{:.2f}'.format
indicadores_petr = ind.todos_indicadores()
print(indicadores_petr)
valuation = ValuationFluxoCaixaDescontado(
            receita_ano=indicadores_petr['ultimareceita'],
            porcenta_crescimento_receita=indicadores_petr['variacaoreceita']['median_deflacionada'] if 'median_deflacionada' in indicadores_petr['variacaoreceita'].keys() else indicadores_petr['variacaoreceita']['median_normal'],
            margem_ebit=indicadores_petr['margemebit'],
            imposto_porcentagem=indicadores_petr['percentualimposto'],
            depreciacao_capex=indicadores_petr['depreciacaocapex'],
            capex_da_receita=indicadores_petr['capexreceita'],
            wacc=indicadores_petr['wacc'],
            numero_de_acoes=indicadores_petr['quantidadeacoes'],
            divida=indicadores_petr['dividatotal'],
            disponivel=indicadores_petr['caixa'],
            ativos_nao_operacionais=indicadores_petr['outrosativos'],
            passivos_circulantes=indicadores_petr['passivosmenosdivida'],
            necessidade_capital_de_giro=indicadores_petr['necessidadecapitalgiro'],
            anos_projecao=5, 
            taxa_crecimento_perpetuidade=0.014,
            calculo_necessidade_capital_de_giro=False)
print(valuation.calcular_valuation())

[*********************100%***********************]  1 of 1 completed


{'margemebit': 0.3882, 'ultimareceita': 102409000000.0, 'variacaoreceita': {'mean_deflacionada': 0.20294717006024565, 'median_deflacionada': 0.40129820010037975, 'mean_normal': 0.28975841631631644, 'median_normal': 0.4824333658861921}, 'depreciacaocapex': 1.6143, 'capexreceita': 0.0932, 'wacc': 0.164, 'quantidadeacoes': 13044496930.0, 'dividatotal': 62600000000.0, 'caixa': 15546000000.0, 'outrosativos': 170536000000.0, 'passivosmenosdivida': 75492000000.0, 'necessidadecapitalgiro': -1415000000.0, 'percentualimposto': 0.2666}
Valuation para 5 anos
Anos 5.00
Valor presente do fluxo de caixa: R$ 1080775988712.18
Fluxo de caixa ajustado: R$ 1191365988712.18
Valor por ação: R$ 91.33
   data     receita_ano        ebit_ano    imposto_ano      capex_ano  \
0  2025 143505547374.08  55708853490.62 14851980340.60 13374717015.26   
1  2026 201094065239.72  78064716126.06 20812053319.21 18741966880.34   
2  2027 281792751671.29 109391946198.79 29163892856.60 26263084455.76   
3  2028 394875675718.

In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format
acao = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()

for acao in acao:
    print(f'Valuation Fluxo de Caixa Descontado da acao {acao}')
    try:
        ind = IndicadoresFinanceiros(ticker=f'{acao}.SA')
        indicadores_petr = ind.todos_indicadores()
        print(indicadores_petr)
        valuation = ValuationFluxoCaixaDescontado(
            receita_ano=indicadores_petr['ultimareceita'],
            porcenta_crescimento_receita=indicadores_petr['variacaoreceita']['median_deflacionada'] if 'median_deflacionada' in indicadores_petr['variacaoreceita'].keys() else indicadores_petr['variacaoreceita']['median_normal'],
            margem_ebit=indicadores_petr['margemebit'],
            imposto_porcentagem=indicadores_petr['percentualimposto'],
            depreciacao_capex=indicadores_petr['depreciacaocapex'],
            capex_da_receita=indicadores_petr['capexreceita'],
            wacc=indicadores_petr['wacc'],
            numero_de_acoes=indicadores_petr['quantidadeacoes'],
            divida=indicadores_petr['dividatotal'],
            disponivel=indicadores_petr['caixa'],
            ativos_nao_operacionais=indicadores_petr['outrosativos'],
            passivos_circulantes=indicadores_petr['passivosmenosdivida'],
            necessidade_capital_de_giro=indicadores_petr['necessidadecapitalgiro'],
            anos_projecao=5, 
            taxa_crecimento_perpetuidade=0.014,
            calculo_necessidade_capital_de_giro=False)
        print(valuation.calcular_valuation())
    except:
        print(f'Erro ao obter dados da acao {acao}')

# Valuation completo

In [31]:
acoes= pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()

In [32]:
acoes[:10]

['AALR3',
 'ABEV3',
 'AFLT3',
 'AGRO3',
 'AGXY3',
 'AHEB3',
 'ALPA3',
 'ALPA4',
 'ALPK3',
 'ALLD3']

In [49]:
pd.read_csv("dados/setor.csv")['tic'].to_list()

['AALR3',
 'ABEV3',
 'AFLT3',
 'AGRO3',
 'AGXY3',
 'AHEB3',
 'ALPA3',
 'ALPA4',
 'ALPK3',
 'ALLD3',
 'ALOS3',
 'AMBP3',
 'AMAR3',
 'AMER3',
 'ANIM3',
 'ASAI3',
 'AURE3',
 'AVLL3',
 'AZEV3',
 'AZUL4',
 'AZZA3',
 'BALM4',
 'BAUH4',
 'BAZA3',
 'BBAS3',
 'BBSE3',
 'BBDC3',
 'BBDC4',
 'BGIP3',
 'BGIP4',
 'BIOM3',
 'BLAU3',
 'BMEB3',
 'BMEB4',
 'BMKS3',
 'BMIN3',
 'BMIN4',
 'BMOB3',
 'BPAN4',
 'BPAC11',
 'BPAC3',
 'BPAC5',
 'BRAP3',
 'BRAP4',
 'BRAV3',
 'BRBI11',
 'BRFS3',
 'BRKM3',
 'BRKM5',
 'BRST3',
 'BRSR3',
 'BRSR6',
 'BSLI3',
 'BSLI4',
 'CAMB3',
 'CAML3',
 'CASH3',
 'CCRO3',
 'CEAB3',
 'CEBR6',
 'CEDO3',
 'CEDO4',
 'CGAS3',
 'CGAS5',
 'CLSA3',
 'CMIG3',
 'CMIG4',
 'COGN3',
 'CRFB3',
 'CSAN3',
 'CSMG3',
 'CSNA3',
 'CSUD3',
 'CYRE3',
 'DIRR3',
 'DXCO3',
 'EALT3',
 'EALT4',
 'ECOR3',
 'EGIE3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENEV3',
 'ENGI11',
 'ENGI3',
 'ENGI4',
 'EQTL3',
 'ESPA3',
 'EZTC3',
 'FESA3',
 'FESA4',
 'FIQE3',
 'FLRY3',
 'FRAS3',
 'GGBR3',
 'GGBR4',
 'GOAU3',
 'GOAU4',
 'GOLL4

In [ ]:
acoes= pd.read_csv("dados/setor.csv")['tic'].to_list()
acoes = acoes[:5]
data_valuation = {
    'acao': [],
    'valor_atual':[],
    'preco_gordon': [],
    'preco_fluxo': [],
    'diferenca_gordon': [],
    'diferenca_fluxo': [],
    'margem_ebit': [],
    'variacao_receita': [],
    'wacc': [],
    'quantidade_acoes': [],
    'divida_total': [],
    'caixa': [],
    'outros_ativos': [],
    'passivos_menos_divida': [],
    'percentual_imposto': [],
}
for acao in acoes:
    print('-'*10, acao, '-'*10)
    try:
        valu = ValuationModoloGordon(f'{acao}.SA')
        preco_gordon = valu.preco_acao()

        ind = IndicadoresFinanceiros(ticker=f'{acao}.SA')
        indicadores = ind.todos_indicadores()

        valuation_fluxo = ValuationFluxoCaixaDescontado(
                    receita_ano=indicadores['ultimareceita'],
                    porcenta_crescimento_receita=indicadores['variacaoreceita']['median_deflacionada'] if 'median_deflacionada' in indicadores['variacaoreceita'].keys() else indicadores['variacaoreceita']['median_normal'],
                    margem_ebit=indicadores['margemebit'],
                    imposto_porcentagem=indicadores['percentualimposto'],
                    depreciacao_capex=indicadores['depreciacaocapex'],
                    capex_da_receita=indicadores['capexreceita'],
                    wacc=indicadores['wacc'],
                    numero_de_acoes=indicadores['quantidadeacoes'],
                    divida=indicadores['dividatotal'],
                    disponivel=indicadores['caixa'],
                    ativos_nao_operacionais=indicadores['outrosativos'],
                    passivos_circulantes=indicadores['passivosmenosdivida'],
                    necessidade_capital_de_giro=indicadores['necessidadecapitalgiro'],
                    anos_projecao=5, 
                    taxa_crecimento_perpetuidade=0.014,
                    calculo_necessidade_capital_de_giro=False)

        _ , valor_fluxo = valuation_fluxo.calcular_valuation()

        yaho = yf.Ticker(f'{acao}.SA')
        valor_acao_atual = round(yaho.history(period="10Y", interval="1d")['Close'][-1],2)
        
        data_valuation['acao'].append(acao)
        data_valuation['valor_atual'].append(valor_acao_atual)
        data_valuation['preco_gordon'].append(preco_gordon['valuation_acao'])
        data_valuation['preco_fluxo'].append(valor_fluxo['valor_por_acao'])
        data_valuation['diferenca_gordon'].append(round((preco_gordon['valuation_acao'] - valor_acao_atual)/valor_acao_atual,2))
        data_valuation['diferenca_fluxo'].append(round((valor_fluxo['valor_por_acao'] - valor_acao_atual)/valor_acao_atual,2))
        data_valuation['margem_ebit'].append(indicadores['margemebit'])
        data_valuation['variacao_receita'].append(indicadores['variacaoreceita'])
        data_valuation['wacc'].append(indicadores['wacc'])
        data_valuation['quantidade_acoes'].append(indicadores['quantidadeacoes'])
        data_valuation['divida_total'].append(indicadores['dividatotal'])
        data_valuation['caixa'].append(indicadores['caixa'])
        data_valuation['outros_ativos'].append(indicadores['outrosativos'])
        data_valuation['passivos_menos_divida'].append(indicadores['passivosmenosdivida'])
        data_valuation['percentual_imposto'].append(indicadores['percentualimposto'])
        
        
        
        
        
    except:
        print(f'Erro ao obter dados da acao {acao}')
data_valores = pd.DataFrame(data_valuation)

In [54]:
data_valores

,acao,valor_atual,preco_gordon,preco_fluxo,diferenca_gordon,diferenca_fluxo,margem_ebit,variacao_receita,wacc,quantidade_acoes,divida_total,caixa,outros_ativos,passivos_menos_divida,percentual_imposto
0,AALR3,10.90,0.13,-3.92,-0.99,-1.36,-0.0080,"{'mean_deflacionada': 0.01780135230773457, 'me...",0.312,1.182928e+08,1.309243e+09,2.348420e+08,1.013896e+09,4.165760e+08,0.1711
1,ABEV3,11.08,6.06,7.05,-0.45,-0.36,0.2194,"{'mean_deflacionada': 0.04142226593848808, 'me...",0.235,1.575766e+10,3.501100e+09,1.633620e+10,6.438510e+10,4.899940e+10,0.0408
2,AFLT3,6.68,2.80,1.14,-0.58,-0.83,0.5708,"{'mean_deflacionada': 0.5005168329959699, 'med...",0.221,6.308470e+07,1.340000e+05,2.936000e+06,5.970000e+05,3.550300e+07,0.0865
3,AGRO3,21.76,14.55,10.35,-0.33,-0.52,0.3743,"{'mean_normal': 0.23323418320771924, 'median_n...",0.143,1.026834e+08,1.043998e+09,1.938940e+08,7.402100e+08,3.814050e+08,0.1029
4,AGXY3,0.66,1.13,NaN,0.71,NaN,0.0810,"{'mean_deflacionada': 0.6459478923550395, 'med...",NaN,2.545436e+08,2.586818e+09,NaN,4.319600e+07,NaN,0.0358


In [14]:
import pandas as pd

# Exemplo de DataFrame
df = pd.read_csv('dados/valores_valuations_acoes.csv')

df_makdown = df.to_markdown(index=False)

with open("dados/valuation.md", "w", encoding="utf-8") as f:
    f.write(df_makdown)


In [ ]:
print(df.to_markdown(index=False))

In [15]:
import pandas as pd

# Lê o arquivo CSV com os dados mais recentes
df = pd.read_csv('dados/valores_valuations_acoes.csv')

# Converte o DataFrame para uma tabela Markdown
markdown_table = df.to_markdown(index=False)

# Abre o arquivo README.md
with open('README.md', 'r') as file:
    readme_content = file.readlines()

# Encontrar a seção "Resultados das Estimativas" e inserir a tabela
for i, line in enumerate(readme_content):
    if '## Resultados das Estimativas' in line:  # Localiza o título da seção
        # Adiciona a tabela logo abaixo do título
        readme_content.insert(i + 1, f"### Tabela de Valuation\n{markdown_table}\n")

# Escreve a versão atualizada no README.md
with open('README.md', 'w') as file:
    file.writelines(readme_content)



In [19]:
markdown_table

"| acao   |   valor_atual |   preco_gordon |   preco_fluxo |   diferenca_gordon |   diferenca_fluxo |   margem_ebit | variacao_receita                                                                                                                                                      |          wacc |   quantidade_acoes |     divida_total |         caixa |    outros_ativos |   passivos_menos_divida |   percentual_imposto |\n|:-------|--------------:|---------------:|--------------:|-------------------:|------------------:|--------------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------:|-------------------:|-----------------:|--------------:|-----------------:|------------------------:|---------------------:|\n| PCAR3  |          2.95 |           2.49 |       4119.87 |              -0.16 |           1395.57 |        0.033  | {'mean_deflacionada': -0.01715

In [17]:
for i, line in enumerate(readme_content):
    if '## Resultados das Estimativas' in line:  # Localiza o título da seção
        # Adiciona a tabela logo abaixo do título
        readme_content.insert(i + 1, f"### Tabela de Valuation\n{markdown_table}\n")

In [18]:
readme_content

['# DescriÃ§Ã£o do RepositÃ³rio\n',
 '\n',
 'Este repositÃ³rio contÃ©m uma aplicaÃ§Ã£o para realizar o valuation de empresas brasileiras utilizando duas metodologias conhecidas:\n',
 '\n',
 '- **Fluxo de Caixa Descontado (DCF):** Avalia o valor da empresa com base na projeÃ§Ã£o e desconto dos fluxos de caixa futuros.\n',
 '- **MÃ©todo de Gordon:** Calcula o valor da empresa considerando dividendos e crescimento perpÃ©tuo.\n',
 '\n',
 'A aplicaÃ§Ã£o integra cÃ¡lculos financeiros e indicadores por meio de mÃ³dulos especÃ\xadficos, permitindo a comparaÃ§Ã£o dos resultados obtidos por ambos os mÃ©todos.\n',
 '\n',
 '## Objetivos\n',
 '\n',
 '- O repositÃ³rio tem como objetivo realizar o valuation de empresas brasileiras.\n',
 '- Integra dados financeiros histÃ³ricos e indicadores obtidos de fontes como o Yahoo Finance.\n',
 '- Permite comparar os resultados dos diferentes mÃ©todos de valuation.\n',
 '- Identificar aÃ§Ãµes baratas de empresas com potencial de crescimento sustentÃ¡vel.\n',
 

In [134]:
import pandas as pd

# Lê o arquivo CSV
df = pd.read_csv('dados/valores_valuations_acoes.csv')

# Converte o DataFrame para tabela em Markdown
markdown_table = df.to_markdown(index=False)

# Abre o arquivo README.md
with open('README.md', 'r') as file:
    readme_content = file.readlines()

# Verifica e localiza a seção "Resultados das Estimativas"
section_found = False
for i, line in enumerate(readme_content):
    if '## Estrutura do RepositÃ³rio' in line:
        index_fim = i
    if '## Resultados das Estimativas' in line:
        index_inicio = i
        
if len(readme_content[index_inicio:index_fim-1]) <= 10:
    print("Tabela não existe ou foi removida.")
    readme_content.insert(index_inicio + 1, f"### Tabela de Valuation\n{markdown_table}\n")
else:
    print("Tabela já existe, substituindo...")
    readme_content = (
        readme_content[:index_inicio + 1]  # Mantém tudo antes da seção
        + ["\n### Tabela de Valuation\n", markdown_table, "\n"]  # Adiciona a nova tabela
        + readme_content[index_fim:]  # Mantém tudo depois da seção
    )
# Escreve a versão atualizada no README.md
with open('README.md', 'w') as file:
    file.writelines(readme_content)  





Tabela não existe ou foi removida.


In [147]:
# Abre o arquivo README.md
with open('README.md', 'r') as file:
    readme_content = file.readlines()

# Verifica e localiza a seção "Resultados das Estimativas"
section_found = False
for i, line in enumerate(readme_content):
    if '## Estrutura do RepositÃ³rio' in line:
        index_fim = i
    if '## Resultados das Estimativas' in line:
        index_inicio = i

readme_content = readme_content[:index_inicio+1] + readme_content[index_fim-1:]
        
readme_content.insert(index_inicio + 1, f"### Tabela de Valuation\n{markdown_table}\n")

with open('README.md', 'w') as file:
    file.writelines(readme_content)

In [148]:
readme_content

['# DescriÃ§Ã£o do RepositÃ³rio\n',
 '\n',
 'Este repositÃ³rio contÃ©m uma aplicaÃ§Ã£o para realizar o valuation de empresas brasileiras utilizando duas metodologias conhecidas:\n',
 '\n',
 '- **Fluxo de Caixa Descontado (DCF):** Avalia o valor da empresa com base na projeÃ§Ã£o e desconto dos fluxos de caixa futuros.\n',
 '- **MÃ©todo de Gordon:** Calcula o valor da empresa considerando dividendos e crescimento perpÃ©tuo.\n',
 '\n',
 'A aplicaÃ§Ã£o integra cÃ¡lculos financeiros e indicadores por meio de mÃ³dulos especÃ\xadficos, permitindo a comparaÃ§Ã£o dos resultados obtidos por ambos os mÃ©todos.\n',
 '\n',
 '## Objetivos\n',
 '\n',
 '- O repositÃ³rio tem como objetivo realizar o valuation de empresas brasileiras.\n',
 '- Integra dados financeiros histÃ³ricos e indicadores obtidos de fontes como o Yahoo Finance.\n',
 '- Permite comparar os resultados dos diferentes mÃ©todos de valuation.\n',
 '- Identificar aÃ§Ãµes baratas de empresas com potencial de crescimento sustentÃ¡vel.\n',
 